In [1]:
import csv
import numpy as np
import os
import pandas as pd
import scipy.interpolate
import sklearn.metrics
import sys
sys.path.append("../src")
import localmodule


if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO


from matplotlib import pyplot as plt
%matplotlib inline

# Define constants.
dataset_name = localmodule.get_dataset_name()
models_dir = localmodule.get_models_dir()
units = localmodule.get_units()
n_units = len(units)
n_trials = 10


In [58]:
# Loop over augmentations.
for aug_kind_str in ["none", "all-but-noise", "all"]:
    
    # Loop over models.
    for model_name in [
            "icassp-convnet", "icassp-ntt-convnet", "icassp-add-convnet",
            "pcen-convnet"]:


        if aug_kind_str == "all-but-noise" and model_name == "icassp-convnet":
            continue

        if not aug_kind_str == "none":
            model_name = "_".join([model_name, "aug-" + aug_kind_str])
        model_dir = os.path.join(models_dir, model_name)

        for test_unit_id in range(6):

            test_unit_str = units[test_unit_id]
            test_unit_dir = os.path.join(model_dir, test_unit_str)

            val_accs = []
            for trial_id in range(n_trials):
                trial_str = "trial-" + str(trial_id)
                trial_dir = os.path.join(test_unit_dir, trial_str)
                history_name = "_".join([
                    dataset_name,
                    model_name,
                    test_unit_str,
                    trial_str,
                    "history.csv"
                ])
                history_path = os.path.join(
                    trial_dir, history_name)
                try:
                    history_df = pd.read_csv(history_path)
                    val_acc = max(history_df["Validation accuracy (%)"])
                except:
                    val_acc = 0.0
                val_accs.append(val_acc)

            val_accs = np.array(val_accs)
            best_trial_id = np.argmi(val_accs)
            best_trial_str = "trial-" + str(best_trial_id)

            validation_units = localmodule.fold_units()[test_unit_id][2]
            predict_folds = [validation_units, [test_unit_str]]

            for predict_fold in predict_folds:
                
                fns, fps, tns, tps, accs = [], [], [], [], []

                for predict_unit_str in predict_fold:

                    best_trial_dir = os.path.join(
                        test_unit_dir, best_trial_str)
                    predictions_name = "_".join([
                        dataset_name,
                        model_name,
                        "test-" + test_unit_str,
                        best_trial_str,
                        "predict-" + predict_unit_str,
                        "clip-predictions.csv"
                    ])
                    prediction_path = os.path.join(
                        best_trial_dir, predictions_name)

                    # Load prediction.
                    with open(prediction_path, 'r') as f:
                        reader = csv.reader(f)
                        rows = list(reader)
                    rows = [",".join(row) for row in rows]
                    rows = rows[1:]
                    rows = "\n".join(rows)

                    # Parse rows with correct header.
                    df = pd.read_csv(StringIO(rows),
                        names=[
                            "Dataset",
                            "Test unit",
                            "Prediction unit",
                            "Timestamp",
                            "Center Freq (Hz)",
                            "Augmentation",
                            "Key",
                            "Ground truth",
                            "Predicted probability"])
                    y_pred = np.array(df["Predicted probability"])
                    y_pred = (y_pred > 0.5).astype('int')

                    # Load ground truth.
                    y_true = np.array(df["Ground truth"])

                    # Compute confusion matrix.
                    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(
                        y_true, y_pred).ravel()

                    tns.append(tn)
                    fps.append(fp)
                    fns.append(fn)
                    tps.append(tp)

                fn = sum(fns)
                fp = sum(fps)
                tn = sum(tns)
                tp = sum(tps)

                print(model_name, "{:5.2f}%".format(100 * (tn+tp) / (tn+tp+fn+fp)))
        
    print("")

icassp-convnet 96.26%
icassp-convnet 50.99%
icassp-convnet 85.19%
icassp-convnet 93.49%
icassp-convnet 84.84%
icassp-convnet 99.10%
icassp-convnet 97.75%
icassp-convnet 96.58%
icassp-convnet 98.26%
icassp-convnet 93.49%
icassp-convnet 96.86%
icassp-convnet 97.29%
icassp-ntt-convnet 96.93%
icassp-ntt-convnet 54.82%
icassp-ntt-convnet 86.95%
icassp-ntt-convnet 94.32%
icassp-ntt-convnet 80.49%
icassp-ntt-convnet 98.60%
icassp-ntt-convnet 97.71%
icassp-ntt-convnet 97.53%
icassp-ntt-convnet 97.98%
icassp-ntt-convnet 92.59%
icassp-ntt-convnet 91.13%
icassp-ntt-convnet 97.20%
icassp-add-convnet 95.90%
icassp-add-convnet 71.02%
icassp-add-convnet 84.11%
icassp-add-convnet 95.43%
icassp-add-convnet 89.43%
icassp-add-convnet 98.94%
icassp-add-convnet 96.82%
icassp-add-convnet 97.04%
icassp-add-convnet 98.58%
icassp-add-convnet 93.01%
icassp-add-convnet 97.11%
icassp-add-convnet 97.96%
pcen-convnet 97.60%
pcen-convnet 74.57%
pcen-convnet 90.76%
pcen-convnet 93.35%
pcen-convnet 85.87%
pcen-convnet

In [33]:
localmodule.fold_units()[test_unit_id][-1]

['unit07', 'unit10']

In [56]:
validation_units

['unit07', 'unit10']